In [1]:
!pip install gtts groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 2.9 MB/s eta 0:00:00


In [2]:
import os
import re

from groq import Groq

from gtts import gTTS
import librosa
import IPython.display as ipd

import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif
import imageio

In [27]:
GROQ_API_KEY = os.environ["GROQ_API_KEY"] = "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"

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY is missing! Make sure it's set in environment variables.")

client = Groq(api_key=GROQ_API_KEY)

# Welcome message
welcome_message = """Welcome to the world of children, where fun and stories abound.
Enjoy this journey."""
print(welcome_message)

story_idea = input("\nMother, please enter the idea for the story: ")

prompt = f"""
IMPORTANT: Write ONLY the story text with no explanations or planning.
 The story must be consistent with the teachings of Islam and not contradict it..

STORY IDEA: {story_idea}

FORMAT: Begin writing the children's story immediately. Include a title and "The End" at the conclusion.
"""

response = client.chat.completions.create(
    model="mixtral-8x7b-32768",
    messages=[
        {"role": "system", "content": "You are a children's storyteller. Write stories directly without explanation or meta-commentary."},
        {"role": "user", "content": prompt}
    ],
    temperature=1,
    max_tokens=1500,
    top_p=1
)

assistant_message = response.choices[0].message.content

if "The End" in assistant_message:
    story_parts = assistant_message.split("The End")
    cleaned_story = story_parts[0].strip() + "\n\nThe End"
    if "**" in cleaned_story:
        story_start = cleaned_story.find("**")
        cleaned_story = cleaned_story[story_start:].strip()
    print("\nHere is the children's story based on the idea:\n")
    print(cleaned_story)
else:
    print("\nHere is the children's story based on the idea:\n")
    print(assistant_message)


Welcome to the world of children, where fun and stories abound.
Enjoy this journey.

Mother, please enter the idea for the story: the god horse

Here is the children's story based on the idea:

Title: The God Horse

In a small village, there lived a young boy named Hassan. He was known for his love of horses. Hassan would spend hours watching and taking care of the village horses. His dream was to own the most magnificent horse in the land.

One day, while Hassan was out in the pasture, he noticed a beautiful horse standing alone. This horse was unlike any Hassan had ever seen before. It had a radiant white coat and a mane as golden as the sun. Its eyes sparkled with an intelligence and kindness that made Hassan feel at ease.

Hassan approached the horse slowly, extending his hand in greeting. To his surprise, the horse lowered its head and allowed Hassan to pet it. From that moment on, Hassan knew that this was no ordinary horse.

Hassan visited the horse every day, bringing it fresh 

In [29]:


def story_to_speech_gtts(story_text, output_file="childrens_story.wav", lang="en"):
    """
    Convert a children's story to speech using Google Text-to-Speech (gTTS).

    Parameters:
    - story_text (str): The text of the story to convert.
    - output_file (str): Filename for the output audio file.
    - lang (str): Language code (default is English: 'en').

    Returns:
    - str: Path to the created audio file.
    """
    tts = gTTS(text=story_text, lang=lang, slow=False)
    tts.save(output_file)

    # Get audio duration
    duration = librosa.get_duration(path=output_file)

    print(f"Audio file saved as: {output_file}, Duration: {duration:.2f} seconds")
    return output_file, duration



audio_path, duration = story_to_speech_gtts(cleaned_story, lang="en")

ipd.display(ipd.Audio(audio_path))


Audio file saved as: childrens_story.wav, Duration: 151.94 seconds


In [5]:
duration

137.76

In [46]:
def extract_scenes(story_text: str, num_scenes: int) -> list:
    prompt_sys = f"""You are a scene extractor and visual describer. Your task is to break a story into exactly {num_scenes} distinct scenes.
    Each scene must capture only one single event, described concisely with a clear visual focus.
    Ensure consistency in descriptions—if an object, place, or character is mentioned multiple times, use the same wording each time.
    If a scene repeats, provide the exact same description as before.
    Avoid merging multiple events into one scene, and do not add extra formatting, punctuation, or numbering."""

    prompt_user = f"""Extract exactly {num_scenes} scenes from the following story.
    Each scene should describe only one clear event, focusing on key visual elements such as setting, characters, and emotions.
    Do not merge multiple events into one scene.
    If an object, place, or character has been named before, mention it explicitly instead of using vague terms.
    If a scene repeats later in the story, ensure it has the same description as before.

    Story: {story_text}

    Provide the scenes as a single string, with each scene separated by '&&'. Do not add numbers, titles, or extra formatting."""

    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": prompt_sys},
            {"role": "user", "content": prompt_user}
        ],
        temperature=1,
        max_tokens=1500,
        top_p=1
    )

    scenes = response.choices[0].message.content
    scenes = scenes.split("&&")

    return scenes


In [41]:

def num_scenes(frame_per_scene=15, fps=6):
  scene_duration = (frame_per_scene / fps)
  num_scenes = int(duration / scene_duration)

  return num_scenes


In [47]:
scenes = extract_scenes(cleaned_story, 60)
len(scenes)

1

In [48]:
type(scenes)

list

In [49]:
scenes

["Scene 1: A small village where a young boy named Hassan lives.\nScene 2: Hassan's love for horses and his habit of taking care of village horses.\nScene 3: Hassan's dream of owning the most magnificent horse in the land.\nScene 4: Hassan spotting a beautiful horse in the pasture.\nScene 5: The horse's radiant white coat and golden mane.\nScene 6: The horse's sparkling, intelligent, and kind eyes.\nScene 7: Hassan approaching the horse slowly.\nScene 8: Hassan extending his hand to greet the horse.\nScene 9: The horse lowering its head and allowing Hassan to pet it.\nScene 10: Hassan's realization that this is no ordinary horse.\nScene 11: Hassan visiting the horse every day.\nScene 12: Hassan bringing fresh water and food for the horse.\nScene 13: The horse allowing Hassan to ride on its back.\nScene 14: Hassan's adventures with the horse.\nScene 15: Hassan hearing a soft, gentle voice.\nScene 16: The voice carrying an authority that Hassan cannot ignore.\nScene 17: The voice address

In [56]:
def clean_and_yield_scenes(scenes: list):
    for text in scenes:
        cleaned_scenes = re.split(r"\nScene \d+:\s*", text)  # Remove "Scene X:"
        for scene in cleaned_scenes:
            if scene.strip():  # Ensure no empty strings
                yield scene.strip()
scene_list = list(clean_and_yield_scenes(scenes))
print(len(scene_list))

60


In [24]:

def generate_t2v_prompt(scene_description: str, retry_times: int = 3) -> str:
    """
    Generates a highly detailed video prompt based on a simple scene description.
    """
    sys_prompt_t2v = """You are part of a team of bots that creates videos. You work with an assistant bot that will draw anything you say in square brackets.

    For example, outputting "a beautiful morning in the woods with the sun peaking through the trees" will trigger your partner bot to output a video of a forest morning, as described. You will be prompted by people looking to create detailed, amazing videos. The way to accomplish this is to take their short prompts and make them extremely detailed and descriptive and so importantly with be consitint with islam.

    There are a few rules to follow:
    - You will only ever output a single video description per user request.
    - When modifications are requested, refactor the entire description to integrate the suggestions.
    - Other times, users may want a completely new video. In this case, ignore previous conversations.
    - Video descriptions must maintain a fixed length based on examples provided below.
    """

    for _ in range(retry_times):
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": sys_prompt_t2v},
                {"role": "user", "content": f'Create a highly detailed video description for: "{scene_description}"'},
            ],
            model="mixtral-8x7b-32768",
            temperature=0.01,
            top_p=0.7,
            stream=False,
            max_tokens=250,
        )

        if response.choices:
            return response.choices[0].message.content

    return scene_description  # Return the original prompt if retries fail

pm = generate_t2v_prompt(scenes[0])
pm

In [60]:
scene_list = [generate_t2v_prompt(scene) for scene in scene_list]

In [17]:

device = "cuda" if torch.cuda.is_available() else "cpu"

adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16 )#, device=device
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.to(device)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")
pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


AnimateLCM_sd15_t2v_lora.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

In [61]:
negative_prompts = "bad quality, worse quality, low resolution, Unfocused faces, Obscured faces, overlay"
all_frames = []

for i, prompt in enumerate(scene_list):
    output = pipe(
        prompt=prompt,
        negative_prompt="bad quality, worse quality, low resolution, Unfocused faces",
        num_frames=15,
        guidance_scale=2.0,
        num_inference_steps=6,
        generator = torch.Generator(device).manual_seed(i),
    )
    frames = output.frames[0]
    all_frames.extend(frames)

    scene_gif_filename = f"scene_{i+1}.gif"
    imageio.mimsave(scene_gif_filename, frames, fps=6)
    torch.cuda.empty_cache()

export_to_gif(all_frames, "full_video.gif")
imageio.mimsave("full_video.mp4", all_frames, fps=6)



The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', the first rays of sunlight peek over the distant mountains, casting a warm, golden hue over the village. the air is filled with the soothing sounds of birds chirping and the gentle rustling of leaves, as the villagers awaken to the promise of a new day. hassan, a curious and energetic young boy, emerges from his humble abode, his eyes sparkling with excitement and wonder. he greets his neighbors with a friendly wave and a bright smile, before setting off on his daily adventures. the village is bustling with activity, as the residents go about their morning routines. women dressed in vibrant, flowing garments can be seen tending to their gardens, their hands deftly plant']


  0%|          | 0/6 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['already awake, his heart filled with anticipation and gratitude for the opportunity to be in the presence of these majestic creatures. he makes his way to the village stable, a spacious, well - ventilated structure made of sturdy timber and adorned with intricate carvings that depict the rich history of the village and its deep connection to horses. as hassan enters the stable, the soft, earthy scent of hay and horseflesh fills the air, mingling with the faint, soothing sounds of horses gently whinnying and shuffling in their stalls. with a tender, knowing touch, hassan greets each horse, whispering words of encouragement and admiration as he checks their water troughs and feed b']


  0%|          | 0/6 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["a young man of unwavering determination and boundless ambition, his eyes alight with the fire of his dreams. his gaze is fixed upon the distant horizon, where the faint outline of a magnificent horse, a creature of unparalleled grace and beauty, beckons him forth. the camera pans across the undulating hills, revealing a tapestry of vibrant hues, as the verdant grasslands blend seamlessly into the azure sky. a gentle breeze whispers through the air, carrying with it the sweet scent of wildflowers, as the delicate petals dance in a symphony of color and motion. as we draw closer to the object of hassan's desires, the magnificent horse comes into full view, a"]


...

  0%|          | 0/6 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["rence and joy as they approach the god horse, their hands gently reaching out to touch its silken mane or the intricate patterns adorning its bridle. the horse, in its wisdom and grace, lowers its head, allowing the villagers to connect with the divine presence that it represents. a gentle breeze rustles through the leaves of the nearby trees, as if whispering a secret message of peace and serenity. the sun's rays filter through the foliage, casting dappled patterns upon the ground and the villagers'upturned faces. the air is filled with the scent of freshly baked bread, the earthy aroma of the soil, and the sweet fragrance of blooming flowers"]


  0%|          | 0/6 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["glow, is a dazzling white, untouched by the impurities of this world. its mane and tail, flowing like a river of silk, are a radiant gold, symbolizing the divine wisdom that guides us through life's challenges. its eyes, deep and profound, are a piercing blue, reflecting the infinite compassion and love that god has for his creation. the horse's build is powerful and majestic, a testament to the strength and resilience that god bestows upon his faithful. its muscles ripple with every movement, a reminder of the unyielding support and guidance that god provides to those who seek his path. its gait is graceful and elegant, a symbol of the beauty and harmony that exists in god's"]


  0%|          | 0/6 [00:00<?, ?it/s]

In [62]:
from moviepy.editor import VideoFileClip, AudioFileClip

video = VideoFileClip("full_video.mp4")
audio = AudioFileClip(audio_path)

video = video.set_audio(audio)

# Export the final video
video.write_videofile("output.mp4", codec="libx264", audio_codec="aac")


Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
